In [103]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tkinter import *
import os

In [113]:
def to_area(area):
    if(area in ['תל אביב','נס ציונה - רחובות',
               'מושבים בשרון','רמת גן - גבעתיים','רמלה - לוד','נתניה והסביבה',
                'רעננה - כפר סבא','הוד השרון והסביבה','בקעת אונו','מושבים במרכז',
               'גדרה יבנה והסביבה','רמת השרון - הרצליה','פתח תקוה והסביבה','ראשל"צ והסביבה',
               'חולון - בת ים']):
        return "center"
    if(area in ['חיפה וחוף הכרמל','טבריה והסביבה','עכו - נהריה','מושבים בצפון',
               'קריות','עמק יזרעאל','גליל ועמקים','חדרה וישובי עמק חפר','זכרון - בנימינה',
               'אזור צפון','כרמיאל והסביבה']):
        return "north"
    if(area in ['ירושלים והסביבה','בית שמש והסביבה',"קיסריה והסביבה",'מושבים בשפלה']):
        return "jerusalem"
    if(area in ['באר שבע והסביבה','אשדוד - אשקלון','מודיעין והסביבה','מושבים בדרום']):
        return "south"
    else: 
        return area

In [114]:
df = pd.read_csv("saved_data.csv").iloc[:,1:]
df = df.drop(["firm","time of ad"],axis=1)
df = df[df.cost != "לא צויין"]
to_cost = lambda x:int(x[0:-2].replace(",",""))
to_int = lambda x : int(x)
df.cost = df.cost.apply(to_cost)
df.km = df.km.apply(to_int)
df["year of prudaction"] = df["year of prudaction"].apply(to_int)
type_dummies = pd.get_dummies(df.car_type).iloc[:,1:]
df = pd.concat([df,type_dummies], axis = 1)
df = df.rename(columns={"רכב פרטי":"private","רכב מסחרי":"commercial","משאיות":"tracks"})
df = df.drop(columns={"car_type"})
df = df[df["km"]<=10**6]
df = df[(df["cost"]<=200_000)&(df["km"]<=600_000)]

In [115]:
    ## note that even though "יישובי השומרון" and "אילת והערבה" are values in "area" we didn't mentioned
    ## them in the to_area function, that's beacause they don't fall to any of the four neat categories
    ## "north","south","center","jerosalem", and each of them has small amount of samples, so to generelize
    ## we will get rid of them, and then apply the to_area function
df = df[(df["area"]!= "יישובי השומרון")&(df["area"]!= "אילת והערבה")]
df["area"] = df["area"].apply(to_area)
type_dummies = pd.get_dummies(df.area).iloc[:,1:]
df = pd.concat([df,type_dummies], axis = 1)
df = df[["cost","km","year of prudaction","tracks","commercial","private","jerusalem","north","south"]]

In [108]:
df.to_csv("saved_data_with_area.csv")

In [116]:
df

,cost,km,year of prudaction,tracks,commercial,private,jerusalem,north,south
0,102000,19000,2017,0,0,1,0,0,0
1,43210,175000,2016,0,0,1,0,0,0
2,70000,129000,2009,0,0,1,0,1,0
3,57000,144000,2012,0,0,1,0,1,0
4,38500,138853,2010,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...
793,142000,177000,2018,1,0,0,0,0,1
794,30000,330000,2006,1,0,0,0,1,0
796,97100,226500,2017,1,0,0,0,0,0
797,68000,560000,2008,1,0,0,0,1,0
